In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sp
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

%matplotlib inline
np.random.seed(2062)
df = pd.read_csv("40403863_features.csv")

In [8]:
#section 4.1
features = list(df.columns[range(2,18)])
features.pop(5)

while True:
    x = sm.add_constant(df[features])
    model = sm.OLS(df["aspect_ratio"], x).fit()
    if model.pvalues.max() < 0.001:
        break;
    else:
        features.remove(model.pvalues.idxmax())
        print("R-Squared", model.rsquared)
print(model.summary())
print("\nFinal R-Squared:", model.rsquared)

R-Squared 0.9461528190109458
R-Squared 0.9461184366438093
R-Squared 0.9459597530482351
R-Squared 0.9451638391822873
R-Squared 0.9429773780422762
R-Squared 0.9413743579013932
R-Squared 0.9381243779841059
R-Squared 0.9341713110472628
                            OLS Regression Results                            
Dep. Variable:           aspect_ratio   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     254.4
Date:                Tue, 11 Mar 2025   Prob (F-statistic):           2.20e-73
Time:                        17:38:55   Log-Likelihood:                 139.70
No. Observations:                 140   AIC:                            -263.4
Df Residuals:                     132   BIC:                            -239.9
Df Model:                           7                                         
Covariance Type:            nonrobust                    

In [2]:
# section 4.2

# most significant is connected_areas

# creating binary column? assignment unclear so I assumed we're still differentiating letters and non-letters
df["letter"] = np.where(df["label"].isin(["a","b","c","d","e","f","g","h","i","j"]), 1, 0)

np.random.seed(42)
sampleSet = df.sample(frac=1, random_state = 34).reset_index(drop=True)
train_data = sampleSet[:112] # top 80% like in topic 14 slides
test_data = sampleSet[112:]

sns.histplot(data=train_data, x="connected_areas", alpha=0.5)
plt.show()
plt.clf()

x_train = train_data[["connected_areas"]]
y_train = train_data["letter"]
model = LogisticRegression()
model.fit(x_train, y_train)
print(model.coef_, model.intercept_)

x_vals = np.linspace(train_data["connected_areas"].min(), train_data["connected_areas"].max(), 1000)
fitted_curve = model.predict_proba(x_vals.reshape(-1, 1))[:, 1]
plt.scatter(train_data["connected_areas"], train_data["letter"], c=train_data["letter"], cmap="viridis", label="Training Data")
plt.plot(x_vals, fitted_curve, label="fitted curve")
plt.show()


train_preds = (model.predict_proba(x_train)[:, 1] > 0.5).astype(int)
train_accuracy = accuracy_score(y_train, train_preds)
print("Training Accuracy:", train_accuracy)

x_test = test_data[["connected_areas"]]
y_test = test_data["letter"]
test_preds = (model.predict_proba(x_test)[:, 1] > 0.5).astype(int)
test_accuracy = accuracy_score(y_test, test_preds)
print("Testing Accuracy:", test_accuracy)

KeyError: 'label'

In [26]:
# section 4.3
splits = [[0,0,0],[0,0,0],[0,0,0]]
df["letter"] = np.where(df["label"].isin(["a","b","c","d","e","f","g","h","i","j"]), 1, 0)
df["face"] = np.where(df["label"].isin(["smiley", "sad"]), 1, 0)
df["xclaim"] = np.where(df["label"] == "xclaim", 1, 0)

types = [df["letter"], df["face"], df["xclaim"]]
for x in range(0,140):
    for y in range(0,3):
        if df["nr_pix"][x] > df["nr_pix"].median() and types[y][x] == 1:
            splits[y][0] += 1
        if df["aspect_ratio"][x] > df["aspect_ratio"].median() and types[y][x] == 1:
            splits[y][1] += 1
        if df["neigh_1"][x] > df["neigh_1"].median() and types[y][x] == 1:
            splits[y][2] += 1
for x in range(0, 3):
    total = splits[0][x] + splits[1][x] + splits[2][x]
    for y in range(0, 3):
        splits[y][x] = round((splits[y][x]/total), 3)
print("Letters: ", splits[0])
print("Faces: ", splits[1])
print("Exclamation Marks: ", splits[2])

Letters:  [0.627, 0.443, 0.184]
Faces:  [0.373, 0.557, 0.776]
Exclamation Marks:  [0.0, 0.0, 0.041]
